In [ ]:
from qdtuner import SingleQuantumDotTuner

In [ ]:
QDTuner = SingleQuantumDotTuner(
    device_config='../device_config.yml',
    station_config='../station_config.yml',
    tuner_config='../tuner_config.yml',
    qcodes_config='../qcodes_config.yml',
    save_dir='C:\\Users\\coher\\Desktop\\Andrija'
)


In [ ]:
QDTuner.bias_device(Vbias=0.005) # 0.5 mV

In [ ]:
# Set maxV equal to None if you want to sweep 
# up to the max voltage you set in device config
# It'll stop if it hits max current or max gate voltage (it should)
QDTuner.check_turn_on(minV=1, maxV=None, dV=0.05)

In [ ]:
QDTuner.device_info

In [ ]:
# Set maxV equal to None if you want to use
# saturation voltage found in turn on phase.
QDTuner.check_pinch_offs(minV=None, maxV=None, dV=0.005)

In [ ]:
# B1 is outer, B2 is inner loop
QDTuner.barrier_barrier_sweep(B1='LB', B2='RB', voltage_configuration={}, dV=0.002, delay=0.01)

In [ ]:
QDTuner.coulomb_blockade(P='P', P_bounds=(0,0.75), voltage_configuration={'S': 0.005, 'LB': 0.55, 'RB': 0.7}, dV=0.005)

In [ ]:
QDTuner.coulomb_diamonds(ohmic='S', 
                         gate='P', 
                         S_bounds=(-0.015, 0.015),
                         gate_bounds=(0,0.75),
                         voltage_configuration={'LB': 0.55, 'RB': 0.7},
                         dV_gate=0.003, 
                         dV_ohmic=0.001)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qcodes as qc

# Get last dataset recorded, convert to current units
dataset = qc.load_last_experiment().last_data_set()
df = dataset.to_pandas_dataframe().reset_index()
df_current = df.copy()
df_current = df_current.rename(columns={f'{QDTuner.multimeter_device}_volt': f'{QDTuner.multimeter_device}_current'})
df_current.iloc[:,-1] = df_current.iloc[:,-1].subtract(QDTuner.preamp_bias).mul(QDTuner.sensitivity) # sensitivity

# Plot 2D colormap
df_pivoted = df_current.pivot_table(values=f'{QDTuner.multimeter_device}_current', index=[f'{QDTuner.voltage_device}_volt_P'], columns=[f'{QDTuner.voltage_device}_volt_S'])
gate_data, ohmic_data = df_pivoted.columns, df_pivoted.index
raw_current_data = df_pivoted.to_numpy()[:,:-1] / 1.0e-9 # convert to nA
gate_grad = np.gradient(raw_current_data, axis=1)
ohmic_grad = np.gradient(raw_current_data, axis=0)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,10))
fig.suptitle("Coulomb Diamonds")

im_ratio = raw_current_data.shape[0]/raw_current_data.shape[1]

cbar_ax1 = plt.colorbar(ax1.imshow(
    raw_current_data,
    extent=[gate_data[0], gate_data[-1], ohmic_data[0], ohmic_data[-1]],
    origin='lower',
    cmap='coolwarm',
    aspect=0.1
), ax=ax1,fraction=0.046, pad=0.04)

cbar_ax1.set_label(r'$I_{SD}$ (nA)')
ax1.set_title(r'$I_{SD}$')
ax1.set_xlabel(r"$V_{{{gate_name}}}$ (V)".format(gate_name='S'))
ax1.set_ylabel(r"$V_{{{gate_name}}}$ (V)".format(gate_name='P'))

# V grad is actually horizontal
grad_vector = (0,1) # Takes the gradient along 45 degree axis

cbar_ax2 = plt.colorbar(ax2.imshow(
    np.sqrt(grad_vector[0] * gate_grad**2 +   grad_vector[1]* ohmic_grad**2),
    extent=[gate_data[0], gate_data[-1], ohmic_data[0], ohmic_data[-1]],
    origin='lower',
    cmap='coolwarm',
    aspect=0.1
), ax=ax2,fraction=0.046, pad=0.04)

cbar_ax2.set_label(r'$\nabla_{\theta=45\circ} I_{SD}$ (nA/V)')
ax2.set_title(r'$\nabla I_{SD}$')
ax2.set_xlabel(r"$V_{{{gate_name}}}$ (V)".format(gate_name='S'))
ax2.set_ylabel(r"$V_{{{gate_name}}}$ (V)".format(gate_name='P'))

fig.tight_layout()

In [ ]:
QDTuner._zero_device()

# DATABASE VIEWER

In [ ]:
db_file = "C:\\Users\\coher\\experiments_2024-04-27.db"
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_file)

In [ ]:
import qcodes as qc
qc.dataset.initialise_or_create_database_at(db_file)
tutorial_exp = qc.dataset.load_or_create_experiment(
    experiment_name="Initialization",
    sample_name="N44_D54"
)
dataset = qc.dataset.load_by_run_spec(experiment_name='Initialization', captured_run_id=1)
